In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",
            'fairness_agg': 'mean',

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 128,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 5,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha': 0.002,
            'lambda': 1.0,
            'lambda_old': 0.0,
              }
    


#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    if params['lambda'] != 0:
        trial_id+=f"_lambda_{params['lambda']}_sigold_{params['lambda_old']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=BiasedMNIST/seed=1_epoch=5_lr=0.001_tau=5_alpha=0.002_lambda_1.0_sigold_0.0


In [2]:
"MNIST" in params['dataset']

True

In [3]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = min(params['per_task_examples'], 15000))

    n_feature = 3*28*28
    

noise = 0.0


[0 1 2 3 4 5 6 7 8 9]


In [4]:
from torchvision.transforms.functional import to_pil_image
from cl_gym.benchmarks.transforms import MNIST_MEAN, MNIST_STD
COLOR_MAP = {
    0: (1, 0, 0),
    1: (0, 1, 0),
    2: (1, 1, 0),
    3: (0, 0, 1),
    4: (1, 0.65, 0),
    5: (0.5, 0, 0.5),
    6: (0, 1, 1),
    7: (1, 0.75, 0.8),
    8: (0.8, 1, 0),
    9: (.588, .294, 0.)
}

r = (1 - 0.1913)
color_values = np.array(list(COLOR_MAP.values()))
m_rgb = color_values.mean(axis=0)
std_rgb = color_values.std(axis=0)

bmnist_mean = [r*m + MNIST_MEAN[0] for m in m_rgb]
bmnist_std = [(r*s**2+(1-r)*MNIST_STD[0]**2+r*(1-r)*(bmnist_mean[i] - m_rgb[i])**2)**0.5 for i, s in enumerate(std_rgb)]

unnormalize = torchvision.transforms.Normalize([-m/s for m, s in zip(bmnist_mean, bmnist_std)], [1/s for s in bmnist_std])
sample_idx = 95
to_pil_image(unnormalize(benchmark.trains[2].inputs[sample_idx]), mode="RGB")

In [5]:
target_label = 0
incremental_step = 1
# cat_img = torch.cat([img for img in benchmark.trains[incremental_step].inputs[benchmark.trains[incremental_step].targets == target_label][20:30]], dim=2)
cat_img = torch.cat([img for img in benchmark.tests[incremental_step].inputs[benchmark.tests[incremental_step].targets == target_label][20:30]], dim=2)
to_pil_image(unnormalize(cat_img), mode="RGB")

In [6]:
from trainers import FairContinualTrainer
from trainers.fair_trainer import FairContinualTrainer2
from metrics import FairMetricCollector
from algorithms.sensitive import Heuristic3

backbone = cl.backbones.MLP2Layers(input_dim=n_feature, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic3(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

trainer = FairContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
if params['fairness_agg'] == "mean":
    agg = np.mean
elif params['fairness_agg'] == "max":
    agg = np.max
else:
    raise NotImplementedError

metric_manager_callback.meters['multiclass_eo'].agg = agg
metric_manager_callback.meters['multiclass_eo'].agg


<function mean at 0x7f80ac12f070>

In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 86.33349815697204, 'loss': 0.0015818076776274552, 'multiclass_eo': [0.142570281124498, 0.3917525773195877], 'accuracy_s0': 100.0, 'accuracy_s1': 73.28385707779572, 'classwise_accuracy': {0: array([909, 980]), 1: array([ 907, 1135])}}
[2] Eval metrics for task 1 >> {'accuracy': 95.86577362222421, 'loss': 0.0006762866221421154, 'multiclass_eo': [0.1254295532646048, 0.03614457831325302], 'accuracy_s0': 100.0, 'accuracy_s1': 91.92129342110711, 'classwise_accuracy': {1: array([1062, 1135]), 0: array([962, 980])}}
[3] Eval metrics for task 1 >> {'accuracy': 96.52207138361952, 'loss': 0.0003872810702233732, 'multiclass_eo': [0.012048192771084376, 0.12371134020618557], 'accuracy_s0': 100.0, 'accuracy_s1': 93.2120233511365, 'classwise_accuracy': {0: array([974, 980]), 1: array([1063, 1135])}}
[4] Eval metrics for task 1 >> {'accuracy': 99.66375977703856, 'loss': 0.0002715977512071037, 'multic

In [9]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.664,  0.   ,  0.   ,  0.   ,  0.   ],
       [94.365, 91.656,  0.   ,  0.   ,  0.   ],
       [93.579, 83.487, 87.1  ,  0.   ,  0.   ],
       [87.42 , 78.23 , 82.282, 82.106,  0.   ],
       [88.753, 76.408, 75.962, 90.069, 86.109]])

In [10]:
[np.round(x, 3) for x in metric_manager_callback.meters['accuracy'].compute_overall()]

[99.664, 93.01, 88.055, 82.51, 83.46]

In [11]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

89.3398468505708

In [12]:
[np.round(x, 3) for x in metric_manager_callback.meters['multiclass_eo'].compute_overall()]

[0.007, 0.114, 0.154, 0.257, 0.194]

In [13]:
np.mean(metric_manager_callback.meters['multiclass_eo'].compute_overall())

0.14518691207380943

In [14]:
task_id = 2

print(f"{task_id=}")
print(f"sensitive samples / all samples = {(benchmark.trains[task_id].sensitive != benchmark.trains[task_id].targets).sum().item()} / {benchmark.trains[task_id].sensitive.shape[0]}")

updated_seq_indices = benchmark.seq_indices_train[task_id]
print(f"sensitive samples / selected samples = {(benchmark.trains[task_id].sensitive[updated_seq_indices] != benchmark.trains[task_id].targets[updated_seq_indices]).sum().item()} / {len(updated_seq_indices)}")


task_id=2
sensitive samples / all samples = 1165 / 12089
sensitive samples / selected samples = 823 / 1262


In [15]:
step_class = 2
for i in range(2, 6):
    incremental_step = i
    print(f"{incremental_step=}")
    one_idx = benchmark.trains[incremental_step].sample_weight > 0.9

    print(f"{2*i-2} : {(benchmark.trains[incremental_step].targets == (2*i-2)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-2)).sum().item()}")
    print(f"{2*i-1} : {(benchmark.trains[incremental_step].targets == (2*i-1)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-1)).sum().item()}")


incremental_step=2
2 : 5958 --> 1653
3 : 6131 --> 1697
incremental_step=3
4 : 5842 --> 1193
5 : 5421 --> 1252
incremental_step=4
6 : 5918 --> 1546
7 : 6265 --> 1553
incremental_step=5
8 : 5851 --> 1451
9 : 5949 --> 1563


In [16]:
for eos in metric_manager_callback.meters['multiclass_eo'].get_data():
    for eo in eos:
        print(round(eo, 3),end=" ")
    print()

0.008 0.005 
0.047 0.13 0.109 0.17 
0.134 0.056 0.173 0.244 0.201 0.119 
0.172 0.183 0.304 0.299 0.233 0.162 0.233 0.467 
0.096 0.174 0.223 0.342 0.138 0.308 0.14 0.163 0.107 0.247 


In [17]:
metric_manager_callback.meters['classwise_accuracy'].get_data()

[{0: array([976, 980]), 1: array([1132, 1135])},
 {1: array([1087, 1135]),
  0: array([911, 980]),
  2: array([ 964, 1032]),
  3: array([ 908, 1010])},
 {0: array([893, 980]),
  1: array([1090, 1135]),
  2: array([ 871, 1032]),
  3: array([ 834, 1010]),
  5: array([720, 892]),
  4: array([918, 982])},
 {1: array([1000, 1135]),
  0: array([850, 980]),
  3: array([ 811, 1010]),
  2: array([ 786, 1032]),
  4: array([852, 982]),
  5: array([694, 892]),
  6: array([850, 958]),
  7: array([ 776, 1028])},
 {1: array([1065, 1135]),
  0: array([820, 980]),
  2: array([ 824, 1032]),
  3: array([ 737, 1010]),
  5: array([653, 892]),
  4: array([773, 982]),
  6: array([887, 958]),
  7: array([ 900, 1028]),
  8: array([854, 974]),
  9: array([ 853, 1009])}]